<a href="https://colab.research.google.com/github/tahaamiri/Facial-Expression-Recognition/blob/main/Copy_of_vggface_emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all we need

In [ ]:
import pathlib
import cv2
import numpy as np
import os
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K
from  keras.callbacks import  ModelCheckpoint, TensorBoard
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Dataset pathes**

In [ ]:
datasetFolderName = './gdrive/MyDrive/VGGFACE/temp_dataset'

train_path=datasetFolderName+'/train/'
validation_path=datasetFolderName+'/validation/'
test_path=datasetFolderName+'/test/'

class_labels = ['anger', 'fear', 'happy', 'sadness', 'surprise']

VGGFACE model implementation


In [ ]:
def model03():
    # Define VGG_FACE_MODEL architecture
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))	
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))

    # Load VGG Face model weights
    model.load_weights('./gdrive/MyDrive/VGGFACE/vgg_face_weights.h5')

    base_model_output = Convolution2D(5, (1,1), name='predictions')(model.layers[-6].output)
    base_model_output = Flatten()(base_model_output)
    base_model_output = Activation('softmax')(base_model_output)

    model = Model(inputs=model.input, outputs=base_model_output)


    return model

**Data generator**

In [ ]:
def list_all_files(directory):
    fs = []
    for root, dirs, files in os.walk(directory):
        for f in files:
            fs.append(os.path.join(root, f))

    return fs
    

class directoryDataGenerator:
    """
    for generating data from Directory
    """

    def __init__(self):
        """
        Set allForTraining True if you want to use all images from all datasets
        if you want to give each of them weights, set to false. Then each dataset
        will have the weight based on "trainWeight" you set in datasets dictionary
        """
        self.shape = 224
        self.class_counts = 5
        # self.classes: A dictionary contraining colors as keyes,
        # and the corresponding serial as valus
        self.dataset = {"path_train": r"./gdrive/MyDrive/VGGFACE/temp_dataset/train/",
                   "path_validation": r"./gdrive/MyDrive/VGGFACE/temp_dataset/validation/"
                   }

        self.labels = self.classSerializer(self.class_counts)

        # Get validation data
        self.trains = self.readyTrain()
        print("Training data listed.")

        # Get training data
        self.validations = self.readyValidation()
        print("Validation data listed.")

        # How many Training images we have in total (We use this to calculate training steps later)
        self.train_images_count = 0
        for x in self.trains.values():
            self.train_images_count += len(x)

    def readyTrain(self):
        """
        Get all images we want to use for training
        """
        dataset_classes = {folderName: list_all_files(os.path.join(self.dataset["path_train"], folderName))
                           for folderName in os.listdir(self.dataset["path_train"]) if os.path.isdir(os.path.join(self.dataset["path_train"], folderName))}
        return dataset_classes

    def readyValidation(self):
        """
        Get all images we want to use for validation
        """
        val_paths = list_all_files(self.dataset["path_validation"])
        return val_paths

    def preProcessImage(self, image):
        if len(image.shape) == 2:
            image = cv2.cvtColor(image , cv2.COLOR_GRAY2RGB)
        # image = tf.keras.applications.vgg16.preprocess_input(image)
        # image = preprocess_input(image, version=2)
        image = cv2.resize(image, (self.shape, self.shape))
        image = image / 255.
        return image

    def classSerializer(self, number_of_classes):
        """
        for every image, we label it as one-hot-encode
        for example  [0,0,0,0,1]
        """
        class_dict = {}
        serial = np.zeros(number_of_classes).astype('float32')
        for c in range(number_of_classes):
            new_serial = serial.copy()
            new_serial[c] = 1.0
            class_dict[c] = new_serial
        return class_dict

    def yieldFromDirectory(self):
            while True:
                for folderName, fileNames in self.trains.items():
                    fName = random.choice(fileNames)
                    image = cv2.imread(fName, 0)
                    if image is None:
                        continue
                    w, h = image.shape
                    # if w < self.shape or h < self.shape:
                    #     continue
                    images = self.preProcessImage(image)
                    if folderName == 'anger':
                      folderName = 0
                    elif folderName == 'fear':
                      folderName = 1
                    elif folderName == 'happy':
                      folderName = 2
                    elif folderName == 'sadness':
                      folderName = 3  
                    elif folderName == 'surprise':
                      folderName = 4
                    yield images, self.labels[int(folderName)]

    def get_batches(self, batch_size):
        Imgs = np.empty([batch_size, self.shape, self.shape, 3])
        Lbls = np.empty([batch_size, len(self.labels)])
        i = 0
        for image, label in self.yieldFromDirectory():
            if i == batch_size:
                shuffleIndices = np.random.permutation(batch_size)
                yield Imgs[shuffleIndices, ...], Lbls[shuffleIndices, ...]
                Imgs, Lbls, i = (np.empty([batch_size, self.shape, self.shape, 3]),
                                 np.empty([batch_size, len(self.labels)]), 0)
            else:
                Imgs[i, ...] = image
                Lbls[i, ...] = label
                i += 1

    def get_data(self):
        Lbls = list()
        Imgs = list()

        for val_path in self.validations:
            image = cv2.imread(val_path)
            if image is None:
                continue
            Imgs.append(self.preProcessImage(image))
            sentiment = (val_path.split("/")[-2])
            if sentiment == 'anger':
              sentiment = 0
            elif sentiment == 'fear':
              sentiment = 1
            elif sentiment == 'happy':
              sentiment = 2
            elif sentiment == 'sadness':
              sentiment = 3  
            elif sentiment == 'surprise':
              sentiment = 4
            Lbls.append(self.labels[sentiment])

        return np.array(Imgs), np.array(Lbls)

**Helper functions**

In [ ]:
def transferBetweenFolders(source, dest, splitRate): 
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in class_labels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', 
                               splitRate)

**Organize file names and class labels in X and Y variables**

In [ ]:
X=[]
Y=[]

def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(class_labels)):
          if(folderName==class_labels[i]):
              Y.append(i)


for i in range(len(class_labels)):
    prepareNameWithLabels(class_labels[i])

X=np.asarray(X)
Y=np.asarray(Y)

**Main**

In [ ]:
BATCH_SIZE = 5*3

modelName = "vgg0_0309"
model = model03()
# model.summary()

for layer in model.layers[:-6]:
        layer.trainable = False

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0

for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    # print("TRAIN:", train_index, "VALIDATION:", val_index)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]

    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(class_labels)):
          if(Y_val[eachIndex]==i):
              classLabel=class_labels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
        
    generator = directoryDataGenerator()
    STEPS = generator.train_images_count // BATCH_SIZE
    STEPS *= 5

    validation_data=generator.get_data()
    
    checkpoint2 = ModelCheckpoint(
    os.path.join(os.getcwd(), 'models', 'val_weights', modelName),
    monitor='val_accuracy', verbose=1, save_best_only=True)

    model.fit(
        generator.get_batches(BATCH_SIZE),
        validation_data=generator.get_data(), 
        epochs=10,
        batch_size=BATCH_SIZE,
        steps_per_epoch=STEPS,
        # validation_steps=50,
        callbacks=[
        checkpoint2
        ],
        shuffle=True)
    
    print('\n')

Results for fold 1
Training data listed.
Validation data listed.
Epoch 1/10
55/55 [==============================] - ETA: 0s - loss: 0.7825 - accuracy: 0.8097
Epoch 1: val_accuracy improved from -inf to 0.54545, saving model to /content/models/val_weights/vgg0_0309
55/55 [==============================] - 21s 364ms/step - loss: 0.7825 - accuracy: 0.8097 - val_loss: 0.9777 - val_accuracy: 0.5455
Epoch 2/10
55/55 [==============================] - ETA: 0s - loss: 0.1413 - accuracy: 0.9952
Epoch 2: val_accuracy improved from 0.54545 to 0.66667, saving model to /content/models/val_weights/vgg0_0309
55/55 [==============================] - 20s 374ms/step - loss: 0.1413 - accuracy: 0.9952 - val_loss: 0.8130 - val_accuracy: 0.6667
Epoch 3/10
55/55 [==============================] - ETA: 0s - loss: 0.0643 - accuracy: 0.9952
Epoch 3: val_accuracy improved from 0.66667 to 0.75758, saving model to /content/models/val_weights/vgg0_0309
55/55 [==============================] - 20s 368ms/step - loss